In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ds-dataset/sensitive_dev.json
/kaggle/input/ds-dataset/sensitive_train.json
/kaggle/input/ds-dataset/sensitive_test.json


In [2]:
import pprint

from sklearn import preprocessing, metrics
import torch
import torchtext
import torchtext.transforms as T
# from torch.hub import load_state_dict_from_url
from torch.utils.data import DataLoader

import pickle

In [3]:
train_df = pd.read_json('/kaggle/input/ds-dataset/sensitive_train.json')
train_df

,text,label
0,Mayorga said that ahead of El Salvador’s legis...,politics
1,"Citing unnamed sources, The Associated Press r...",politics
2,"Instead, she said, Bukele came to meet with th...",politics
3,Mayorga said Bukele’s government didn’t say an...,politics
4,Animosity between the two appears to have star...,politics
...,...,...
1527,Why do mass casualty attackers Livestream thei...,terrorism
1528,Paris Attacks Trial Resumes With Main Suspect ...,terrorism
1529,"4 Burkina Troops, 3 Civilians Killed in Jihadi...",terrorism
1530,Why is ISIS recruiting beyond countries in phy...,terrorism


In [4]:
train_df['label'].unique()

array(['politics', 'religion', 'terrorism', 'neutral', 'insult'],
      dtype=object)

In [5]:
label_encoder = preprocessing.LabelEncoder()
train_df['label'] = label_encoder.fit_transform(train_df['label'])
train_df

,text,label
0,Mayorga said that ahead of El Salvador’s legis...,2
1,"Citing unnamed sources, The Associated Press r...",2
2,"Instead, she said, Bukele came to meet with th...",2
3,Mayorga said Bukele’s government didn’t say an...,2
4,Animosity between the two appears to have star...,2
...,...,...
1527,Why do mass casualty attackers Livestream thei...,4
1528,Paris Attacks Trial Resumes With Main Suspect ...,4
1529,"4 Burkina Troops, 3 Civilians Killed in Jihadi...",4
1530,Why is ISIS recruiting beyond countries in phy...,4


In [6]:
train_df['label'].unique()

array([2, 3, 4, 1, 0])

In [7]:
test_df = pd.read_json('/kaggle/input/ds-dataset/sensitive_test.json')
test_df

,text,label
0,"ET, saying that the GOP failed to show that an...",politics
1,"Despite potential safety concerns, Gates has b...",politics
2,The word cheating also trended on Twitter on t...,politics
3,The worker is seen telling voters to place the...,politics
4,Both Lake and Ward have espoused baseless clai...,politics
...,...,...
1014,After MEPs passed a scathing Russian resolutio...,politics
1015,Kim Jong Un 'Seriously Ill' In North Korea Cov...,politics
1016,Israeli rules say West Bank visitors must decl...,politics
1017,Ethiopia Deploys New Troops into Neighboring S...,politics


In [8]:
test_df['label'].unique()

array(['politics', 'neutral', 'insult', 'religion', 'terrorism'],
      dtype=object)

In [9]:
test_df['label'] = label_encoder.transform(test_df['label'])
test_df

,text,label
0,"ET, saying that the GOP failed to show that an...",2
1,"Despite potential safety concerns, Gates has b...",2
2,The word cheating also trended on Twitter on t...,2
3,The worker is seen telling voters to place the...,2
4,Both Lake and Ward have espoused baseless clai...,2
...,...,...
1014,After MEPs passed a scathing Russian resolutio...,2
1015,Kim Jong Un 'Seriously Ill' In North Korea Cov...,2
1016,Israeli rules say West Bank visitors must decl...,2
1017,Ethiopia Deploys New Troops into Neighboring S...,2


In [10]:
test_df['label'].unique()

array([2, 1, 0, 3, 4])

In [11]:
# https://pytorch.org/text/main/transforms.html#padtransform
from torch import Tensor

class PadTransform(torch.nn.Module):
    """Pad tensor to a fixed length with given padding value.

    :param max_length: Maximum length to pad to
    :type max_length: int
    :param pad_value: Value to pad the tensor with
    :type pad_value: bool
    """

    def __init__(self, max_length: int, pad_value: int) -> None:
        super().__init__()
        self.max_length = max_length
        self.pad_value = float(pad_value)
        
    def forward(self, x: Tensor) -> Tensor:
        """
        :param x: The tensor to pad
        :type x: Tensor
        :return: Tensor padded up to max_length with pad_value
        :rtype: Tensor
        """
        max_encoded_length = x.size(-1)
        if max_encoded_length < self.max_length:
            pad_amount = self.max_length - max_encoded_length
            x = torch.nn.functional.pad(x, (0, pad_amount), value=self.pad_value)
        return x

In [12]:
padding_idx = 1
bos_idx = 0
eos_idx = 2
max_seq_len = 256

In [13]:
# https://pytorch.org/text/main/tutorials/sst2_classification_non_distributed.html#
text_transform = torchtext.models.XLMR_LARGE_ENCODER.transform()

100%|██████████| 5.07M/5.07M [00:00<00:00, 5.20MB/s]
Downloading: "https://download.pytorch.org/models/text/xlmr.vocab.pt" to /root/.cache/torch/hub/checkpoints/xlmr.vocab.pt


  0%|          | 0.00/4.85M [00:00<?, ?B/s]

In [14]:
print(train_df.iloc[0, 0])
text_transform(train_df.iloc[0, 0])

Mayorga said that ahead of El Salvador’s legislative elections, Torres spread false information by saying Bukele had come to Washington to meet with administration officials


[0,
 104610,
 208,
 2804,
 450,
 99255,
 111,
 540,
 61356,
 26,
 7,
 62289,
 13,
 81843,
 7,
 4,
 113984,
 93403,
 98320,
 4677,
 390,
 54433,
 667,
 49713,
 1902,
 1380,
 47,
 17955,
 47,
 23356,
 678,
 86052,
 51521,
 7,
 2]

In [15]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        return (
            # PadTransform(max_length=256, pad_value=padding_idx)(torch.tensor(text_transform(self.df.iloc[idx, 0]))),
            torch.tensor(text_transform(self.df.iloc[idx, 0])),
            torch.nn.functional.one_hot(torch.tensor(self.df.iloc[idx, 1]), 5)
            # torch.tensor(self.df.iloc[idx, 1])
        )

In [16]:
train_dataset = CustomDataset(train_df)
# pprint.pprint(next(iter(train_dataset)))

In [17]:
test_dataset = CustomDataset(test_df)
# pprint.pprint(next(iter(test_dataset)))

In [18]:
# https://stackoverflow.com/questions/35050753/how-big-should-batch-size-and-number-of-epochs-be-when-fitting-a-model#:~:text=Generally%20batch%20size%20of%2032,b%2Fw%2050%20to%20100.
# https://stackoverflow.com/questions/68479235/cuda-out-of-memory-error-cannot-reduce-batch-size
SMALL_BATCH_SIZE = 16
K = 2
LARGE_BATCH_SIZE = K * SMALL_BATCH_SIZE

In [19]:
import torchtext.functional as F

def batch_collate_fn(batch):
    inp_list = list()
    tar_list = list()
    
    for sample in batch:
        inp_list.append(sample[0].tolist())
        tar_list.append(sample[1])
        
    padded_tensor = F.to_tensor(inp_list, padding_value=padding_idx)
    target_tensor = torch.stack(tar_list).double()
    
    return padded_tensor, target_tensor

In [20]:
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=SMALL_BATCH_SIZE,
    shuffle=True,
    collate_fn=batch_collate_fn
)

In [21]:
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=SMALL_BATCH_SIZE,
    shuffle=False,
    collate_fn=batch_collate_fn
)

In [22]:
# next(iter(train_dataloader)) 

In [23]:
num_classes = 5
input_dim = 768

from torchtext.models import RobertaClassificationHead, XLMR_BASE_ENCODER

classifier_head = RobertaClassificationHead(num_classes=num_classes, input_dim=input_dim)
model = XLMR_BASE_ENCODER.get_model(head=classifier_head)

Downloading: "https://download.pytorch.org/models/text/xlmr.base.encoder.pt" to /root/.cache/torch/hub/checkpoints/xlmr.base.encoder.pt


  0%|          | 0.00/1.03G [00:00<?, ?B/s]

In [24]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
DEVICE

device(type='cuda')

In [25]:
model.to(DEVICE)

RobertaModel(
  (encoder): RobertaEncoder(
    (transformer): TransformerEncoder(
      (token_embedding): Embedding(250002, 768, padding_idx=1)
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (dropout): Dropout(p=0.1, inplace=False)
          (attention): MultiheadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (input_projection): Linear(in_features=768, out_features=2304, bias=True)
            (output_projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (residual_mlp): ResidualMLP(
            (mlp): Sequential(
              (0): Linear(in_features=768, out_features=3072, bias=True)
              (1): GELU()
              (2): Dropout(p=0.1, inplace=False)
              (3): Linear(in_features=3072, out_features=768, bias=True)
              (4): Dropout(p=0.1, inplace=False)
            )
          )
          (attention_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)


In [26]:
import torchtext.functional as F
from torch.optim import AdamW

learning_rate = 1.4e-5  # 1e-5  
optim = AdamW(model.parameters(), lr=learning_rate)

# e-3 = default is really bad (predict only one class) ,
# e-7 is too slow, e-6 is too slow
# e-5 by far has the fastest converges, but not consistently there
# 3e-6 isnt good
# 5e-6 converges slower but also good (might be better than e-5 ?)
# 6e-6 is good
# 7e-6 is good (consistently ~0.75 acc, reached ~0.76 acc)
# 8e-6 is nearly the best (more consistent ~0.75 acc, reached ~0.76 acc)
# 9e-6 is nearly the best
# 1.1e-5 is good
# 1.2e-5 is the best
# 1.3e-5 testing
# 1.4e-5 testing

criteria = torch.nn.CrossEntropyLoss()


def train_step(input, target, small_batch_no):
    output = model(input)
    loss = criteria(output, target)
    loss.backward()
    
    if (small_batch_no + 1) % K == 0 or (small_batch_no + 1) == len(train_dataloader):
        optim.step()
        optim.zero_grad()


def evaluate():
    model.eval()
    
    total_loss = 0
    counter = 0
    
    with torch.no_grad():
        # for batch in dev_dataloader:
        for i, batch in enumerate(test_dataloader):
            input = batch[0].clone().detach().to(DEVICE)
            output = model(input)
            target = batch[1].clone().detach().to(DEVICE)
            
            if i == 0:
                class_output = torch.argmax(output, dim=1)
                class_target = torch.argmax(target, dim=1)
            else:
                class_output = torch.cat([class_output, torch.argmax(output, dim=1)])
                class_target = torch.cat([class_target, torch.argmax(target, dim=1)])
            
            # loss, predictions, step_confusion_matrix = eval_step(input, target)
            # loss = eval_step(input, target)
            loss = criteria(output, target).item()
            
            total_loss += loss
            # correct_predictions += predictions
            # total_predictions += len(target)
            counter += 1
            
            
        confusion_matrix = metrics.confusion_matrix(
            class_target.cpu().numpy().flatten(),
            class_output.cpu().numpy().flatten(),
            labels=[0, 1, 2, 3, 4]
        ) 
        classification_report = metrics.classification_report(
            class_target.cpu().numpy().flatten(),
            class_output.cpu().numpy().flatten(),
            labels=[0, 1, 2, 3, 4],
            output_dict=True
        )
            
    

    return (
        total_loss,
        counter,
        confusion_matrix,
        classification_report
    )



In [27]:
num_epochs = 50

In [28]:
def log_epoch(epoch, **kwargs):
    with open(f'log_epoch_{epoch}.pkl', 'wb') as f:
        pickle.dump(kwargs, f)
    
    # the model is really heavy (1.11GB)
    # so only save each 5-th one and the last one
    if epoch % 5 == 0 or epoch == num_epochs - 1:
        torch.save(model.state_dict(), f'model_epoch_{epoch}.pth')

In [29]:
for e in range(num_epochs):
    # wonder if i need this or not
    model.train()
    
    for small_batch_no, small_batch in enumerate(train_dataloader):
        # input = F.to_tensor(batch["token_ids"], padding_value=padding_idx).to(DEVICE)
        # target = torch.tensor(batch["target"]).to(DEVICE)
        # input = F.to_tensor(batch[0], padding_value=padding_idx).to(DEVICE)
        input = small_batch[0].clone().detach().to(DEVICE)
        target = small_batch[1].clone().detach().to(DEVICE)
        train_step(input, target, small_batch_no)
        
        torch.cuda.empty_cache()

    total_loss, counter, confusion_matrix, classification_report = evaluate()
    # loss = evaluate()
    print(f'EPOCH {e}')
    print(f'total_loss={total_loss}')
    print(f'counter={counter}')
    print(f'loss=total_loss/counter={total_loss/counter}')
    print(f'confusion_matrix=\n{confusion_matrix}')
    
    print('classification_report=')
    pprint.pprint(classification_report)
    
    log_epoch(
        e,
        total_loss=total_loss,
        counter=counter,
        loss=total_loss/counter,
        confusion_matrix=confusion_matrix,
        classification_report=classification_report
    )
    # print("loss = [{}], \nconfusion_matrix = [{}], \nclassification_report = [{}]"\
    #      .format(loss, confusion_matrix, classification_report))
    

/opt/conda/lib/python3.7/site-packages/torch/_jit_internal.py:1134: UserWarning: The inner type of a container is lost when calling torch.jit.isinstance in eager mode. For example, List[int] would become list and therefore falsely return True for List[float] or List[str].
  warnings.warn("The inner type of a container is lost when "
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/skle

EPOCH 0
total_loss=91.27679351073775
counter=64
loss=total_loss/counter=1.4261998986052773
confusion_matrix=
[[ 56 129   0   0  20]
 [  0 153   0   0 117]
 [  0   8   0   0 166]
 [  0 121   0   0  38]
 [  0   3   0   0 208]]
classification_report=
{'0': {'f1-score': 0.42911877394636017,
       'precision': 1.0,
       'recall': 0.2731707317073171,
       'support': 205},
 '1': {'f1-score': 0.4473684210526315,
       'precision': 0.3695652173913043,
       'recall': 0.5666666666666667,
       'support': 270},
 '2': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 174},
 '3': {'f1-score': 0.0, 'precision': 0.0, 'recall': 0.0, 'support': 159},
 '4': {'f1-score': 0.5473684210526315,
       'precision': 0.3788706739526412,
       'recall': 0.985781990521327,
       'support': 211},
 'accuracy': 0.40922473012757604,
 'macro avg': {'f1-score': 0.2847711232103246,
               'precision': 0.3496871782687891,
               'recall': 0.36512387777906213,
               'support'

/opt/conda/lib/python3.7/site-packages/torch/_jit_internal.py:1134: UserWarning: The inner type of a container is lost when calling torch.jit.isinstance in eager mode. For example, List[int] would become list and therefore falsely return True for List[float] or List[str].
  warnings.warn("The inner type of a container is lost when "


EPOCH 1
total_loss=70.64861022240736
counter=64
loss=total_loss/counter=1.103884534725115
confusion_matrix=
[[102  79   2   4  18]
 [  1 150  38   4  77]
 [  0   4  65   0 105]
 [  0  66   0  82  11]
 [  0   3   0   0 208]]
classification_report=
{'0': {'f1-score': 0.6623376623376623,
       'precision': 0.9902912621359223,
       'recall': 0.4975609756097561,
       'support': 205},
 '1': {'f1-score': 0.5244755244755246,
       'precision': 0.4966887417218543,
       'recall': 0.5555555555555556,
       'support': 270},
 '2': {'f1-score': 0.4659498207885304,
       'precision': 0.6190476190476191,
       'recall': 0.3735632183908046,
       'support': 174},
 '3': {'f1-score': 0.6586345381526104,
       'precision': 0.9111111111111111,
       'recall': 0.5157232704402516,
       'support': 159},
 '4': {'f1-score': 0.6603174603174603,
       'precision': 0.4964200477326969,
       'recall': 0.985781990521327,
       'support': 211},
 'accuracy': 0.5956820412168793,
 'macro avg': {'f1-sc

/opt/conda/lib/python3.7/site-packages/torch/_jit_internal.py:1134: UserWarning: The inner type of a container is lost when calling torch.jit.isinstance in eager mode. For example, List[int] would become list and therefore falsely return True for List[float] or List[str].
  warnings.warn("The inner type of a container is lost when "


...

EPOCH 47
total_loss=115.10689880164318
counter=64
loss=total_loss/counter=1.7985452937756747
confusion_matrix=
[[170  31   0   2   2]
 [  1 148  81  26  14]
 [  0  15 111   3  45]
 [  0  37   0 121   1]
 [  0   2   4   1 204]]
classification_report=
{'0': {'f1-score': 0.9042553191489361,
       'precision': 0.9941520467836257,
       'recall': 0.8292682926829268,
       'support': 205},
 '1': {'f1-score': 0.588469184890656,
       'precision': 0.6351931330472103,
       'recall': 0.5481481481481482,
       'support': 270},
 '2': {'f1-score': 0.6,
       'precision': 0.5663265306122449,
       'recall': 0.6379310344827587,
       'support': 174},
 '3': {'f1-score': 0.7756410256410258,
       'precision': 0.7908496732026143,
       'recall': 0.7610062893081762,
       'support': 159},
 '4': {'f1-score': 0.8553459119496855,
       'precision': 0.7669172932330827,
       'recall': 0.966824644549763,
       'support': 211},
 'accuracy': 0.7399411187438666,
 'macro avg': {'f1-score': 0.74474

/opt/conda/lib/python3.7/site-packages/torch/_jit_internal.py:1134: UserWarning: The inner type of a container is lost when calling torch.jit.isinstance in eager mode. For example, List[int] would become list and therefore falsely return True for List[float] or List[str].
  warnings.warn("The inner type of a container is lost when "


EPOCH 48
total_loss=117.56322756165827
counter=64
loss=total_loss/counter=1.8369254306509104
confusion_matrix=
[[159  41   1   4   0]
 [  0 134  94  33   9]
 [  0  13 116   6  39]
 [  0  24   0 135   0]
 [  0   3   8   2 198]]
classification_report=
{'0': {'f1-score': 0.8736263736263736,
       'precision': 1.0,
       'recall': 0.775609756097561,
       'support': 205},
 '1': {'f1-score': 0.5525773195876289,
       'precision': 0.6232558139534884,
       'recall': 0.4962962962962963,
       'support': 270},
 '2': {'f1-score': 0.5903307888040712,
       'precision': 0.5296803652968036,
       'recall': 0.6666666666666666,
       'support': 174},
 '3': {'f1-score': 0.7964601769911505,
       'precision': 0.75,
       'recall': 0.8490566037735849,
       'support': 159},
 '4': {'f1-score': 0.8665207877461708,
       'precision': 0.8048780487804879,
       'recall': 0.9383886255924171,
       'support': 211},
 'accuracy': 0.7281648675171737,
 'macro avg': {'f1-score': 0.735903089351079,
 

/opt/conda/lib/python3.7/site-packages/torch/_jit_internal.py:1134: UserWarning: The inner type of a container is lost when calling torch.jit.isinstance in eager mode. For example, List[int] would become list and therefore falsely return True for List[float] or List[str].
  warnings.warn("The inner type of a container is lost when "


EPOCH 49
total_loss=128.34153404941736
counter=64
loss=total_loss/counter=2.0053364695221463
confusion_matrix=
[[172  28   1   2   2]
 [  1 114 113  28  14]
 [  0   8 119   5  42]
 [  0  22   0 136   1]
 [  0   2   7   1 201]]
classification_report=
{'0': {'f1-score': 0.9100529100529099,
       'precision': 0.9942196531791907,
       'recall': 0.8390243902439024,
       'support': 205},
 '1': {'f1-score': 0.5135135135135135,
       'precision': 0.6551724137931034,
       'recall': 0.4222222222222222,
       'support': 270},
 '2': {'f1-score': 0.5748792270531401,
       'precision': 0.49583333333333335,
       'recall': 0.6839080459770115,
       'support': 174},
 '3': {'f1-score': 0.8217522658610273,
       'precision': 0.7906976744186046,
       'recall': 0.8553459119496856,
       'support': 159},
 '4': {'f1-score': 0.8535031847133757,
       'precision': 0.7730769230769231,
       'recall': 0.95260663507109,
       'support': 211},
 'accuracy': 0.7281648675171737,
 'macro avg': {'f1